In [1]:
import numpy as np
import trimesh
import tqdm 

In [2]:
def uniform_sampling_from_mesh(vertices, faces, sample_num):
    # -------- TODO -----------
    # 1. compute area of each triangles
    # 2. compute probability of each triangles from areas
    # 3. sample N faces according to the probability
    # 4. for each face, sample 1 point
    # Note that FOR-LOOP is not allowed!
    # -------- TODO -----------
    # vertices: [N, 3], faces: [N, 3]

    # calcuate area of each triangle
    v0 = vertices[faces[:, 0], :]
    v1 = vertices[faces[:, 1], :]
    v2 = vertices[faces[:, 2], :]
    area = 0.5 * np.linalg.norm(np.cross(v1 - v0, v2 - v0), axis=1)

    # compute probability of each triangles from areas
    prob = area / np.sum(area)

    # sample N faces according to the probability
    face_sample_idx = np.random.choice(faces.shape[0], size=sample_num, p=prob)

    # for each face, sample 1 point
    a1 = np.random.rand(sample_num, 1)
    a2 = np.random.rand(sample_num, 1)
    uniform_pc = (1 - np.sqrt(a1)) * v0[face_sample_idx] + np.sqrt(a1) * (1 - a2) * v1[face_sample_idx] + np.sqrt(a1) * a2 * v2[face_sample_idx]

    return area, prob, uniform_pc
        

In [3]:
def farthest_point_sampling(pc, sample_num):
    # -------- TODO -----------
    # FOR LOOP is allowed here.
    # -------- TODO -----------

    idx = np.random.randint(0, pc.shape[0])
    results = np.array([pc[idx]])
    pc = np.delete(pc, idx, axis=0)
    

    # (k, 1, 3) compared to (1, n-k, 3) -> distance:(k,n-k)
    
    for _ in tqdm.tqdm(range(sample_num-1)):
        distance = np.min(np.linalg.norm(results[:,None] - pc[None], axis=-1), axis=0)
        idx = np.argmax(distance)
        results = np.concatenate([results, pc[idx][None]])
        # delete the point that has been selected
        pc = np.delete(pc, idx, axis=0)

    return results

In [4]:
# task 1: uniform sampling 

obj_path = 'spot.obj'
mesh = trimesh.load(obj_path)
print('faces shape: ', mesh.faces.shape)
sample_num = 512
area, prob, uniform_pc = uniform_sampling_from_mesh(mesh.vertices, mesh.faces, sample_num)


# Visualization. For you to check your code
np.savetxt('uniform_sampling_vis.txt', uniform_pc)

print('area shape: ',area.shape)
print('prob shape: ',prob.shape)
print('pc shape: ',uniform_pc.shape)
# the result should satisfy: 
#       area.shape = (13712, ) 
#       prob.shape = (13712, ) 
#       uniform_pc.shape = (512, 3) 

# For submission
save_dict = {'area': area, 'prob': prob, 'pc': uniform_pc}
np.save('../results/uniform_sampling_results', save_dict)

faces shape:  (13712, 3)
area shape:  (13712,)
prob shape:  (13712,)
pc shape:  (512, 3)


In [5]:
# task 2: FPS

init_sample_num = 2000
final_sample_num = 512
_,_, tmp_pc = uniform_sampling_from_mesh(mesh.vertices, mesh.faces, init_sample_num)
fps_pc = farthest_point_sampling(tmp_pc, final_sample_num)

# Visualization. For you to check your code
np.savetxt('fps_vis.txt', fps_pc)

# For submission
np.save('../results/fps_results', fps_pc)

100%|██████████| 511/511 [00:05<00:00, 98.23it/s] 


In [14]:
# task 3: metrics

from earthmover.earthmover import earthmover_distance
# EMD may be very slow (1~2mins)
# -----------TODO---------------
# compute chamfer distance and EMD for two point clouds sampled by uniform sampling and FPS.
# sample and compute CD and EMD again. repeat for five times.
# save the mean and var.
# -----------TODO---------------

# for 5 times
CD, EMD = [], []
for _ in tqdm.tqdm(range(5)):
    _, _, uniform_samples = uniform_sampling_from_mesh(mesh.vertices, mesh.faces, final_sample_num)
    _, _, tmp_uniform_samples = uniform_sampling_from_mesh(mesh.vertices, mesh.faces, init_sample_num)
    fps_samples = farthest_point_sampling(tmp_uniform_samples, final_sample_num)
    
    cd_distance = (np.mean(np.min(np.linalg.norm(uniform_samples[:,None] - fps_samples[None], axis=-1), axis=0)) + \
    np.mean(np.min(np.linalg.norm(fps_samples[:,None] - uniform_samples[None], axis=-1), axis=0))) / 2
    CD.append(cd_distance)

    # change nparray (512, 3) to tuples
    emd_distance = earthmover_distance(tuple(map(tuple, uniform_samples)), tuple(map(tuple, fps_samples)))
    EMD.append(emd_distance)

CD_mean = np.mean(CD)
CD_var = np.var(CD)
EMD_mean = np.mean(EMD)
EMD_var = np.var(EMD)

# For submission
np.save('../results/metrics', {'CD_mean':CD_mean, 'CD_var':CD_var, 'EMD_mean':EMD_mean, 'EMD_var':EMD_var})

 20%|██        | 1/5 [00:10<00:41, 10.43s/it]

move 0.001953125 dirt from (9.894840502312443, 35.21773789262301, 40.778440336715605) to (9.039202435770166, 35.024413645032546, 40.285349075479445) for a cost of 0.0019654200485241186
move 0.001953125 dirt from (29.74904496454979, 28.143718426563986, 20.95359851432495) to (29.091570123417874, 29.364244606408665, 21.546774851634105) for a cost of 0.0029451515632521684
move 0.001953125 dirt from (30.772942946232728, 23.447871271694904, 20.21727256660266) to (29.048127950942686, 24.362840404589473, 20.342650422738746) for a cost of 0.0038212803969325136
move 0.001953125 dirt from (18.792705604737357, 49.99549313810904, 21.352529728947708) to (18.570761588315136, 49.87070537105309, 20.52797317312159) for a cost of 0.0016854966848112393
move 0.001953125 dirt from (22.4187848324082, 18.426467751891096, 35.54118738912385) to (21.899870410868946, 19.77502742273076, 35.56481727129891) for a cost of 0.0028225486003980622
move 0.001953125 dirt from (40.054574036355035, 14.50166814712668, 26.1522

 40%|████      | 2/5 [00:21<00:31, 10.64s/it]

move 0.001953125 dirt from (47.34969778553648, 33.47406565766435, 34.85855268765695) to (47.178762037158926, 33.56502844131438, 34.8201637931593) for a cost of 0.000385547890660808
move 0.001953125 dirt from (29.5240979488826, 37.372864965122126, 28.13760589316845) to (30.622954971870946, 37.38295424994385, 28.355653613245334) for a cost of 0.002188139344757363
move 0.001953125 dirt from (53.15467595387458, 21.196296977661124, 33.46388733693264) to (52.99060392357126, 21.05193065089822, 34.187920438259646) for a cost of 0.0014771426388638183
move 0.001953125 dirt from (10.197619151161192, 43.418884234848015, 26.114713652034112) to (9.926129140376723, 41.40456998080968, 24.592453000496622) for a cost of 0.004959724811486763
move 0.001953125 dirt from (6.748503012466414, 35.59617619758172, 36.35989461537971) to (6.938561042133038, 36.371849878281544, 37.8124457348981) for a cost of 0.0032375345874163064
move 0.001953125 dirt from (7.771027484086398, 39.97273730888463, 28.568667418379498)

100%|██████████| 511/511 [00:05<00:00, 92.55it/s]


move 0.001953125 dirt from (17.603136827707903, 46.87917350152224, 19.305528886056578) to (16.72037615848128, 48.95840301036007, 20.34357401653726) for a cost of 0.004855391536709966
move 0.001953125 dirt from (12.177026057113578, 48.34027153326506, 26.826582031069016) to (19.009777191136426, 42.81293152149055, 39.89517028663918) for a cost of 0.030759453950537006
move 0.001953125 dirt from (19.743892066216443, 28.326088560107685, 28.392214344055052) to (19.820471017722994, 27.99546364937345, 28.290875505968344) for a cost of 0.0006917667876092983
move 0.001953125 dirt from (43.21309069656426, 14.024126207791824, 36.33788820795819) to (42.58431147091535, 14.380516410284349, 35.91565399084514) for a cost of 0.0016348706576797577
move 0.001953125 dirt from (10.237621781808734, 43.49983952532763, 26.071757082676154) to (9.771029145125855, 43.3625074227755, 28.65154559769752) for a cost of 0.005127419092379481
move 0.001953125 dirt from (30.868976809188776, 14.685015835300453, 31.900774090

 80%|████████  | 4/5 [00:43<00:10, 10.91s/it]

move 0.001953125 dirt from (17.93836605671888, 40.21276249632746, 22.585881161195978) to (19.89695269009355, 40.10922304621275, 22.96470120484655) for a cost of 0.003901504267399422
move 0.001953125 dirt from (26.300071699532918, 16.342863534191512, 20.197750271230593) to (26.195453537263873, 16.345800112289627, 20.19796838318007) for a cost of 0.00020441327255496807
move 0.001953125 dirt from (29.934213907403873, 14.86664099589928, 31.35758213761256) to (29.033425590592277, 15.001022611159211, 31.48134900877307) for a cost of 0.0017951718412521907
move 0.001953125 dirt from (53.76123544530893, 23.120094564660043, 32.888910395763766) to (53.38823903048779, 21.579168191418226, 31.056916300973576) for a cost of 0.004731959880174982
move 0.001953125 dirt from (21.895635123508868, 46.55874577589139, 38.55245812306159) to (21.792210138171903, 46.97288338402514, 37.36328675157771) for a cost of 0.002467698454616439
move 0.001953125 dirt from (45.68752055763483, 8.041256887740733, 27.18532697

100%|██████████| 5/5 [00:53<00:00, 10.76s/it]

move 0.001953125 dirt from (23.616049594083055, 11.746733086271709, 27.794288698576267) to (25.91668005512118, 12.650340166865323, 29.04067109871431) for a cost of 0.005406620912290213
move 0.001953125 dirt from (52.44985672472918, 18.93457109045951, 34.711861790546024) to (53.21501776428471, 21.63615802471338, 33.90830608172716) for a cost of 0.005704243641950667
move 0.001953125 dirt from (15.056794006826877, 30.524097212539402, 33.560385358762545) to (13.285825598805967, 30.409509797065272, 33.76029602773167) for a cost of 0.00348807764967442
move 0.001953125 dirt from (38.29994475733035, 19.716672530187147, 20.83819185939896) to (39.24928523795191, 20.5710562827014, 20.860873310525086) for a cost of 0.002494908597974647
move 0.001953125 dirt from (14.841876319212021, 50.814632656846264, 35.086508368073254) to (14.201262504274865, 51.000315319522116, 33.77094935494767) for a cost of 0.0028808160935061087
move 0.001953125 dirt from (16.331300926492833, 53.441564026171264, 25.43916358

In [16]:
print(CD_mean, CD_var, EMD_mean, EMD_var)

1.3793029672542412 0.00045374637638367465 2.3015443397009827 0.012937756369968556
